In [ ]:
!pip install timm -q
!pip install fastai --upgrade --q
!pip install fastaudio --q
exit()

     |████████████████████████████████| 548 kB 36.9 MB/s 
     |████████████████████████████████| 163 kB 69.7 MB/s 
     |████████████████████████████████| 183 kB 31.0 MB/s 
     |████████████████████████████████| 1.9 MB 47.0 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 194 kB 11.8 MB/s 
     |████████████████████████████████| 804.1 MB 2.8 kB/s 
     |████████████████████████████████| 19.1 MB 1.2 MB/s 
     |████████████████████████████████| 19.1 MB 1.1 MB/s 
     |████████████████████████████████| 21.0 MB 1.3 MB/s 
     |████████████████████████████████| 23.2 MB 1.5 MB/s 
     |████████████████████████████████| 23.3 MB 1.4 MB/s 
     |████████████████████████████████| 23.3 MB 1.4 MB/s 
     |████████████████████████████████| 22.1 MB 1.2 MB/s 
     |████████████████████████████████| 22.1 MB 1.3 MB/s 
     |████████████████████████████████| 17.4 MB 919 kB/s 
     |████████████████████████████████| 1.6 MB 50.5 MB/s 
ERROR: pip's de

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%time
!unzip -o -q '/content/drive/MyDrive/swahili-audio-classification/Swahili_words.zip' -d "/content/Swahili_words"

CPU times: user 84.6 ms, sys: 17.8 ms, total: 102 ms
Wall time: 13.8 s


In [ ]:
import warnings
from fastai.vision.all import *
from fastaudio.core.all import *
from fastaudio.augment.all import *
import numpy as np

path = "/content/drive/MyDrive/swahili-audio-classification/"
warnings.filterwarnings('ignore')
seed = 2020

/usr/local/lib/python3.7/dist-packages/torchaudio/backend/utils.py:47: UserWarning: "torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE" flag is deprecated and will be removed in 0.9.0. Please remove the use of flag.
  '"torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE" flag is deprecated and will be removed in 0.9.0. '


In [ ]:
train = pd.read_csv(path+"Train.csv")
test = pd.read_csv(path+"Test.csv")
ss = pd.read_csv(path+"SampleSubmission.csv")

train

,Word_id,Swahili_word,English_translation
0,id_v8rz06e6rv31.wav,mbili,two
1,id_vmbwicdpfn68.wav,tatu,three
2,id_injlouhxg1hg.wav,ndio,yes
3,id_gdq23p6xgzya.wav,nne,four
4,id_lpstklz51zpz.wav,nane,eight
...,...,...,...
4195,id_ho8aqa3s42zp.wav,hapana,no
4196,id_3lhkgf9dqr4c.wav,kumi,ten
4197,id_ixb7vv4i0zct.wav,tano,five
4198,id_7tboov3kavg4.wav,nne,four


In [ ]:
from sklearn.model_selection import StratifiedKFold

train["Word_id"] = "/content/Swahili_words/" + train["Word_id"]
test["Word_id"] = "/content/Swahili_words/" + test["Word_id"]

skf = StratifiedKFold(10, random_state=seed, shuffle=True)
for fold, (_, val_inx) in enumerate(skf.split(train, train.Swahili_word)):
    train.loc[val_inx, 'fold'] = fold
train.fold = train.fold.astype(int)
train

,Word_id,Swahili_word,English_translation,fold
0,/content/Swahili_words/id_v8rz06e6rv31.wav,mbili,two,0
1,/content/Swahili_words/id_vmbwicdpfn68.wav,tatu,three,8
2,/content/Swahili_words/id_injlouhxg1hg.wav,ndio,yes,4
3,/content/Swahili_words/id_gdq23p6xgzya.wav,nne,four,6
4,/content/Swahili_words/id_lpstklz51zpz.wav,nane,eight,0
...,...,...,...,...
4195,/content/Swahili_words/id_ho8aqa3s42zp.wav,hapana,no,4
4196,/content/Swahili_words/id_3lhkgf9dqr4c.wav,kumi,ten,8
4197,/content/Swahili_words/id_ixb7vv4i0zct.wav,tano,five,3
4198,/content/Swahili_words/id_7tboov3kavg4.wav,nne,four,3


In [ ]:
def random_seed(seed_value, use_cuda): 
   np.random.seed(seed_value)
   torch.manual_seed(seed_value) 

   random.seed(seed_value)

   if use_cuda: 
      torch.cuda.manual_seed(seed_value) 
      torch.cuda.manual_seed_all(seed_value) 

      torch.backends.cudnn.deterministic = True 

      torch.backends.cudnn.benchmark = False 

random_seed(seed, True)
set_seed(seed, True)

In [ ]:
augs = [
        AddNoiseGPU, ChangeVolumeGPU, DeltaGPU, MaskFreqGPU, MaskTimeGPU, SignalCutoutGPU, SignalLossGPU, TfmResizeGPU
]

In [ ]:
params = {'n_fft':2048, 'hop_length':308, 'n_mels':224, 'sample_rate':16000, 'win_length':1024}
cfg = AudioConfig.BasicMelSpectrogram(**params) # pull up the voice config
aud2spec = AudioToSpec.from_cfg(cfg) # let audio to spec use the config

In [ ]:
class AudioNormalize(Transform):
    "Normalizes a single `AudioTensor`."
    def encodes(self, x:AudioTensor): return (x-x.mean()) / x.std()

def get_dls(fold):
    torch.cuda.empty_cache()
    audio = DataBlock(blocks=(AudioBlock(force_mono=True, crop_signal_to=5000), 
                            CategoryBlock),
                        splitter=MaskSplitter(train.fold == fold),
                        get_x = ColReader(0),
                        get_y=ColReader(1),
                        item_tfms = [aud2spec, AudioNormalize],
                        batch_tfms = [RemoveSilence, ChangeVolume, SignalLoss ]        
        )
    dls = audio.dataloaders(train, bs=64, num_workers=0, device='cuda')
    dls.rng.seed(seed)

    return dls

In [ ]:
predictions = []
for fold in [4]:
    print(f"\n\nFold {fold}")
    dls = get_dls(fold)
    model_name = f'/content/fold{fold}-model'
    learn = cnn_learner(dls, resnet50, CrossEntropyLossFlat(), metrics=accuracy, n_in=1)
    callbacks = [
             SaveModelCallback(monitor='valid_loss', fname=model_name) 
             ]
    learn.unfreeze()
    lr = learn.lr_find(show_plot=False)[0]
    learn.fit_one_cycle(16, wd=0.1, lr_max=lr, cbs=callbacks)


    tdl = learn.dls.test_dl(test["Word_id"]) # quicly create test data loader

    model = learn.load(model_name)
    test_preds_tta, test_labels_tta = model.tta(dl=tdl, n=8)
    predictions.append(test_preds_tta)



Fold 4


epoch,train_loss,valid_loss,accuracy,time
0,2.184472,2.184595,0.600000,01:34
1,1.058665,1.444458,0.692857,01:33
2,0.723173,2.609138,0.626190,01:34
3,0.598250,2.147221,0.576190,01:33
4,0.444309,2.291647,0.616667,01:34
5,0.387562,0.915799,0.780952,01:34
6,0.310271,0.577832,0.804762,01:34
7,0.333647,1.371626,0.669048,01:34
8,0.307194,0.425286,0.892857,01:34
9,0.206452,0.309532,0.907143,01:33


Better model found at epoch 0 with valid_loss value: 2.1845946311950684.
Better model found at epoch 1 with valid_loss value: 1.4444580078125.
Better model found at epoch 5 with valid_loss value: 0.9157994389533997.
Better model found at epoch 6 with valid_loss value: 0.5778323411941528.
Better model found at epoch 8 with valid_loss value: 0.425286203622818.
Better model found at epoch 9 with valid_loss value: 0.30953237414360046.
Better model found at epoch 10 with valid_loss value: 0.15822184085845947.
Better model found at epoch 12 with valid_loss value: 0.1346624791622162.
Better model found at epoch 13 with valid_loss value: 0.11354061216115952.


In [ ]:
ss[ss.columns[1:]] = np.mean([i.numpy() for i in predictions], axis=0)
ss.head()

,Word_id,hapana,kumi,mbili,moja,nane,ndio,nne,saba,sita,tano,tatu,tisa
0,id_jp2pxl0r84ya.wav,4.721344e-07,4.004166e-07,2.633734e-06,2.179727e-07,2.124520e-07,1.558306e-06,2.910615e-05,1.062045e-05,9.998897e-01,5.806639e-07,5.455514e-07,0.000064
1,id_ndduqqvthbpx.wav,5.653394e-09,7.290681e-09,1.920394e-08,3.571269e-09,5.690303e-08,2.694652e-08,1.589246e-08,7.531496e-08,3.220496e-06,6.893893e-09,1.084257e-08,0.999997
2,id_36oxymxfcm6q.wav,2.158750e-03,8.593222e-03,8.101210e-03,3.223895e-02,8.682068e-03,1.305359e-02,2.990446e-03,2.301702e-02,3.070341e-01,2.140534e-03,2.227786e-02,0.569712
3,id_ue9b0to760pg.wav,7.549601e-05,5.526344e-06,6.201109e-06,1.074014e-05,2.994733e-05,5.404312e-06,3.561699e-06,2.787708e-04,1.211216e-07,9.995755e-01,2.682789e-06,0.000006
4,id_prja4oprb914.wav,5.714251e-02,4.773939e-02,5.965301e-02,3.148311e-02,5.236311e-01,3.157707e-02,4.175781e-02,2.183136e-02,2.241595e-02,3.436878e-02,7.519815e-02,0.053202


In [ ]:
from sklearn.metrics import log_loss 

val = pd.read_csv(path+'Validation.csv')[['Word_id', 'Swahili_word']]
chk = ss[ss.Word_id.apply(lambda x:x in val.Word_id.unique())]

log_loss(val.Swahili_word, chk[chk.columns[1:]])

0.08296884353019936

In [ ]:
name = "/content/sub2.csv"
ss.to_csv(name, index=False)